#            HashTag Recommendation based on user's post 

Importing required libraries

In [104]:
import pandas as pd
import seaborn as sns
%matplotlib inline
import numpy as np
from gensim.models import Word2Vec
import warnings
warnings.filterwarnings('ignore')
from  nltk.tokenize  import  TweetTokenizer 
import glob
import nltk
import string
import re
import pprint
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

Reading all the csv files to create a corpus

In [179]:
allFiles = glob.glob("Data/*/*.csv", recursive=True)
frame = pd.DataFrame()

# To store filenames
filenames = []


for file_ in allFiles:    
    df = pd.read_csv(file_, encoding = "ISO-8859-1")
    filenames.append(file_.split("\\")[-1:][0].split(".")[0])
#     print(df.head())
    frame = pd.concat([frame,df[df.columns[1]]])

In [180]:
print(allFiles)

['Data\\FACup.csv']


Converting the corpus to lower case

In [181]:
def preprocessing(frame):
    frame = frame.apply(lambda x: x.astype(str).str.lower())
    frame = frame.apply(lambda x: x.astype(str).str.replace('b\'',''))
    frame = frame.apply(lambda x: x.astype(str).str.replace('b\"',''))
    return frame

In [182]:
frame =  preprocessing(frame)

In [183]:
print(frame.head())

                                                   0
0   #facup rt @d_degea: wembley is so special. am...
1   #facup rt @d_degea: wembley is so special. am...
2   #facup rt @chrissmalling: true character to c...
3   #facup rt @d_degea: wembley is so special. am...
4   #facup rt @d_degea: wembley is so special. am...


Function to build stopwords to remove them

In [184]:
def build_stopwords(): 
    stop_words = (stopwords.words('english'))

    list_remove = ['.','/','b\'','b\"','xe2','x80','xa6','x99t','x99s','x87','xb4','xaa','x9a','x9b','x93','xb2','x8c','x9c','xb5','xab']

    stop_words = stop_words +  word_tokenize(string.punctuation) + list_remove

    return stop_words
#     print(stop_words)

Removing stop words and building a corpus

In [185]:
corpus = []
tknzr  =  TweetTokenizer () 
stop_words = build_stopwords()

for row in frame[frame.columns[0]]:
#     print (row)
#     tweet = []    
#     tweet.append( )
#     print (tweet)
#     Removing urls which can be noise
    row = re.sub(r'https?:\/\/.*[\r\n]*', '', row)
    word_tokens = tknzr.tokenize(row)
#     Removing stop words
    filtered_sentence = [w for w in word_tokens if not w in stop_words]    
    corpus.append(filtered_sentence)

In [186]:
print (corpus[:10])

[['#facup', 'rt', '@d_degea', 'wembley', 'special', 'amazing', 'atmosphere', 'great', 'game', '...', 'final', 'xf0', 'x9f', 'x92', 'xf0', 'x9f', 'x8f', 'xbb', 'xf0', 'x9f', 'x94', 'n', 'nwembley', 'es', 'especial', 'gran', 'ambiente'], ['#facup', 'rt', '@d_degea', 'wembley', 'special', 'amazing', 'atmosphere', 'great', 'game', '...', 'final', 'xf0', 'x9f', 'x92', 'xf0', 'x9f', 'x8f', 'xbb', 'xf0', 'x9f', 'x94', 'n', 'nwembley', 'es', 'especial', 'gran', 'ambiente'], ['#facup', 'rt', '@chrissmalling', 'true', 'character', 'come', 'behind', 'make', 'sure', 'matter', 'get', 'another', 'fa', 'cup', 'final', 'proud', 'f'], ['#facup', 'rt', '@d_degea', 'wembley', 'special', 'amazing', 'atmosphere', 'great', 'game', '...', 'final', 'xf0', 'x9f', 'x92', 'xf0', 'x9f', 'x8f', 'xbb', 'xf0', 'x9f', 'x94', 'n', 'nwembley', 'es', 'especial', 'gran', 'ambiente'], ['#facup', 'rt', '@d_degea', 'wembley', 'special', 'amazing', 'atmosphere', 'great', 'game', '...', 'final', 'xf0', 'x9f', 'x92', 'xf0', 'x

Building a word2vec model

In [187]:
import timeit
start_time = timeit.default_timer()

model = Word2Vec(corpus, size=1000, window=3, min_count=1, workers=4)

elapsed = timeit.default_timer() - start_time

print(elapsed)

0.6757893854519352


To validate the results from model

In [190]:
print('Number of words in vocab = ' + str(len(model.wv.vocab)))
# print('Similarity between woman and man = ' + str(model.wv.similarity('man', 'trump')))
# 
# pprint.pprint(model.most_similar("support"))


Number of words in vocab = 648


Input by user to recommend hashtags

In [198]:
input = 'man utd victory fa cup cup win tottenham football'

# input = 'tonight is the book reading sunday holiday night'

In [199]:
''' This function takes input post from the user and suggests hashtags which he can most likely relate his post with
    It recommends hashtags based on current trending topics and post content, 
    not just with the meaning of words but also the 
    context of the words is taken into consideration
'''
def getHashTagSuggestion(input):
#     Tokenizing the input
    input = word_tokenize(input)
#     Getting a list of hashtags from the data collected
    file_inputs = {}
    for file in filenames:
        file = '#' + file.lower()
        file_inputs[file.lower()] = file
    
#     Iterating to find similaity between each words of input and all hashtags
    scores_out = []
    for out in file_inputs.keys():
        list_out = []
        if(out in model.wv.vocab):
            for word in input:    
#             print(word)
#             print(out)
                if(word in model.wv.vocab): 
#                 pprint.pprint(word)
                    list_out.append(model.similarity(word,out))
#                 print(model.similarity(word,out))

        if(len(list_out) > 0):
            scores_out.append((out,sum(list_out) / float(len(list_out))))
#     Returns sorted order of hashtag suggestion score
    scores_out = sorted(scores_out, key=lambda tup: tup[1],reverse = True)
    return scores_out

In [200]:
print (file_inputs)

{'#420day': '#420day', '#alboe': '#ALBOE', '#busphsymposia': '#BUSPHsymposia', '#dev4peace': '#dev4peace', '#earthday18': '#EarthDay18', '#enoughisenough': '#EnoughIsEnough', '#flashbackfriday': '#FlashbackFriday', '#foodfri': '#FoodFri', '#fridayfeeling': '#FridayFeeling', '#garyveebreakfast': '#GaryVeeBreakfast', '#greatreadpbs': '#GreatReadPBS', '#helpcopyafilm': '#HelpCopyAFilm', '#nationallookalikeday': '#NationalLookAlikeDay', '#nationalschoolwalkout': '#NationalSchoolWalkout', '#ncaagym': '#NCAAGym', '#newmusicfriday': '#NewMusicFriday', '#notearslefttocry': '#NoTearsLeftToCry', '#ringstruck4': '#RingStruck4', '#scifrilive': '#SciFriLive', '#signsyoureinmyhometown': '#SignsYoureInMyHometown', '#deepwaterhorizon': '#DeepwaterHorizon', '#four20': '#Four20', '#funfactfriday': '#FunFactFriday', '#higpa': '#HIGPA', '#labcsummit18': '#LABCSummit18', '#rstatsnyc': '#rstatsnyc', '#ucdavisgiveday': '#UCDavisGiveDay', '#universalfancon': '#UniversalFanCon', '#bbcaq': '#bbcaq', '#bbcyoungm

In [173]:
model.similarity('utd','#facup')

0.983841470513196

In [171]:
# scores_out = []

Displaying top HashTags related to user's post

In [203]:
HashTagSuggestion = []
suggestions = getHashTagSuggestion(input)
for out_tuple in suggestions[:5]:
    HashTagSuggestion.append(file_inputs[out_tuple[0]])
    
pprint.pprint (HashTagSuggestion)

[('#facup', 0.9790136429028582)]


Defining a HashTag - These words define what a particular HastTag means

In [134]:
defining_words = model.most_similar("#book",topn = 10)
pprint.pprint ([x[0] for x in defining_words[:10]])


['#artwork',
 'formula',
 'viable',
 'emeritus',
 '#maidenhead',
 '#raccoon',
 'tem',
 'milk',
 'katelyn-kyla-peng',
 'cobos']


In [160]:
print(model.wv.vocab)

{'#facup': <gensim.models.keyedvectors.Vocab object at 0x0000023F822EC898>, 'rt': <gensim.models.keyedvectors.Vocab object at 0x0000023F822E6F98>, '@d_degea': <gensim.models.keyedvectors.Vocab object at 0x0000023F8264CA90>, 'wembley': <gensim.models.keyedvectors.Vocab object at 0x0000023F85DA44A8>, 'special': <gensim.models.keyedvectors.Vocab object at 0x0000023F822D9198>, 'amazing': <gensim.models.keyedvectors.Vocab object at 0x0000023F822D9080>, 'atmosphere': <gensim.models.keyedvectors.Vocab object at 0x0000023F822D93C8>, 'great': <gensim.models.keyedvectors.Vocab object at 0x0000023F822D9438>, 'game': <gensim.models.keyedvectors.Vocab object at 0x0000023F822D97F0>, '...': <gensim.models.keyedvectors.Vocab object at 0x0000023F822D9B38>, 'final': <gensim.models.keyedvectors.Vocab object at 0x0000023F822D9BA8>, 'xf0': <gensim.models.keyedvectors.Vocab object at 0x0000023F822D94A8>, 'x9f': <gensim.models.keyedvectors.Vocab object at 0x0000023F822D90F0>, 'x92': <gensim.models.keyedvecto

The MIT License (MIT)

Copyright (c) 2016-2017 Rohan Jahagirdar, Sai Nikhil Dogiparty

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.